In [10]:
import pandas as pd
import httpx
from selectolax.parser import HTMLParser
from bs4 import BeautifulSoup
import json

In [11]:
def save_dict_to_json(dictionary, path):
    with open(path, 'w') as file:
        json.dump(dictionary, file)

In [7]:
url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm?passo=0'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}

resp = httpx.get(url, headers=headers)

dict_uffici = {}
errors=[]
if resp.status_code == 200:
    html = HTMLParser(resp.text)
    uffici = html.css("option")
    for ufficio in uffici:
        value = ufficio.attributes.get('value')
        dict_uffici[value] = ufficio.text().strip()
else:
    print('Not entered')


dict_uffici

{None: '',
 'C55': 'ABBIATEGRASSO',
 'TMD': 'ABBIATEGRASSO  UT DPMI1',
 'D84': 'ACIREALE',
 'TXD': 'ACIREALE  UT DPCT',
 'F17': 'ACQUI TERME',
 'TSD': 'ACQUI TERME  UT DPAL',
 'F85': 'ADRIA',
 'T5D': 'ADRIA  UT DPRO',
 'D70': 'AGRIGENTO',
 'TXE': 'AGRIGENTO  UT DPAG',
 'D51': 'AGROPOLI',
 'TED': 'AGROPOLI  UT DPSA',
 'F27': 'ALBA',
 'TSE': 'ALBA  UT DPCN',
 'E59': 'ALBANO LAZIALE',
 'TJD': 'ALBANO LAZIALE  UT DPRM3',
 'C36': 'ALBENGA',
 'F13': 'ALESSANDRIA',
 'TSF': 'ALESSANDRIA  UT DPAL',
 'A01': 'ANCONA',
 'TQD': 'ANCONA  UT DPAN',
 'A21': 'AOSTA',
 'T4D': 'AOSTA  UT DRAO',
 'B81': 'AREZZO',
 'TZD': 'AREZZO UT DPAR',
 'D34': 'ARIANO IRPINO',
 'TEE': 'ARIANO IRPINO  UT DPAV',
 'F39': 'ARONA',
 'TSG': 'ARONA  UT DPNO',
 'F99': 'ARZIGNANO',
 'T5E': 'ARZIGNANO  UT DPVI',
 'A06': 'ASCOLI PICENO',
 'TQE': 'ASCOLI PICENO  UT DPAP',
 'F22': 'ASTI',
 'TSH': 'ASTI  UT DPAT',
 'A41': 'ATRI',
 'TAD': 'ATRI  UT DPTE',
 'E14': 'AUGUSTA',
 'TXF': 'AUGUSTA  UT DPSR',
 'B99': 'AULLA',
 'TZE': 'AULLA 

In [12]:
data = [{"code": key, "office": value} for key, value in dict_uffici.items()]
uffici_finanziari = pd.DataFrame(data)

uffici_finanziari.to_excel('Resources/uffici_finanziari.xlsx', index=False)


In [2]:
import json
import os
from selectolax.parser import HTMLParser
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains


In [21]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
lista_uffici = uffici['office'].str.lower().tolist()
len(lista_uffici)

['abbiategrasso',
 'abbiategrasso  ut dpmi1',
 'acireale',
 'acireale  ut dpct',
 'acqui terme',
 'acqui terme  ut dpal',
 'adria',
 'adria  ut dpro',
 'agrigento',
 'agrigento  ut dpag',
 'agropoli',
 'agropoli  ut dpsa',
 'alba',
 'alba  ut dpcn',
 'albano laziale',
 'albano laziale  ut dprm3',
 'albenga',
 'alessandria',
 'alessandria  ut dpal',
 'ancona',
 'ancona  ut dpan',
 'aosta',
 'aosta  ut drao',
 'arezzo',
 'arezzo ut dpar',
 'ariano irpino',
 'ariano irpino  ut dpav',
 'arona',
 'arona  ut dpno',
 'arzignano',
 'arzignano  ut dpvi',
 'ascoli piceno',
 'ascoli piceno  ut dpap',
 'asti',
 'asti  ut dpat',
 'atri',
 'atri  ut dpte',
 'augusta',
 'augusta  ut dpsr',
 'aulla',
 'aulla  ut dpms',
 'avellino',
 'avellino  ut dpav',
 'aversa',
 'aversa  ut dpce',
 'avezzano',
 'avezzano  ut dpaq',
 'badia polesine',
 'badia polesine  ut dpro',
 'bagheria',
 'bagheria  ut dppa',
 'barcellona pozzo di gotto',
 'barcellona pozzo di gotto  ut dpme',
 'bari  ut dpba',
 'bari 1',
 'bari

In [34]:
len(lista_uffici)

725

In [39]:
def click_button(driver, button):
    try:
        button.click()
    except Exception as e:
        driver.execute_script("arguments[0].click();", button) 



 
def fetch_uffici(driver, comune):
    lista = []
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#ufficio.form-control")))
        button = driver.find_element(By.CSS_SELECTOR, "select#ufficio.form-control")
        button.send_keys(comune)
        button.send_keys(Keys.ENTER)


        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input#avanti.btn.btn-primary")))
        button = driver.find_element(By.CSS_SELECTOR, "input#avanti.btn.btn-primary")
        click_button(driver,button)
    except:
        print('Non riuscito')


    if driver.find_element(By.CSS_SELECTOR, "select#ente.form-control"):

        button = driver.find_element(By.CSS_SELECTOR, "select#ente.form-control")
        click_button(driver,button)
        all_option = button.find_elements(By.CSS_SELECTOR,'option')

        for option in all_option:
            lista.append(option.text)

    else:
        print('Not entered in if.')
    return lista
    




 
def main():
    errors = list() 
    comuni_list = lista_uffici.copy()
    options = webdriver.EdgeOptions()
    driver = webdriver.Edge(options=options)
    driver.maximize_window()
    url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm?passo=0'
    dict_option = {}
 
    chunks = [
        comuni_list[0:150],
        comuni_list[150:300],
        comuni_list[300:450],
        comuni_list[450:600],
        
    ]
 
    for i, chunk in enumerate(chunks):
        dict_option.clear()
        for j,comune in enumerate(chunk):
            print(j,' Processing comune:', comune)
            driver.get(url)
            tribunali = fetch_uffici(driver, comune)
            if tribunali == []:
                errors.append(comune)
            else: 
                dict_option[comune] = tribunali

        save_dict_to_json(dict_option, f'Resources/enti_{(i+1)*150}.json')
        with open(f"Errors/errors_enti_{(i+1)*150}.txt", "w") as file:
            for item in errors:
                file.write(item + "\n")
    driver.quit()

 
if __name__ == "__main__":
    main()

0  Processing comune: abbiategrasso
1  Processing comune: abbiategrasso  ut dpmi1
2  Processing comune: acireale
3  Processing comune: acireale  ut dpct
4  Processing comune: acqui terme
5  Processing comune: acqui terme  ut dpal
6  Processing comune: adria
7  Processing comune: adria  ut dpro
8  Processing comune: agrigento
9  Processing comune: agrigento  ut dpag
10  Processing comune: agropoli
11  Processing comune: agropoli  ut dpsa
12  Processing comune: alba
13  Processing comune: alba  ut dpcn
14  Processing comune: albano laziale
15  Processing comune: albano laziale  ut dprm3
16  Processing comune: albenga
17  Processing comune: alessandria
18  Processing comune: alessandria  ut dpal
19  Processing comune: ancona
20  Processing comune: ancona  ut dpan
21  Processing comune: aosta
22  Processing comune: aosta  ut drao
23  Processing comune: arezzo
24  Processing comune: arezzo ut dpar
25  Processing comune: ariano irpino
26  Processing comune: ariano irpino  ut dpav
27  Process

In [3]:
def merge_json_files(folder_path):
    merged_dict = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                merged_dict.update(json_data)
    return merged_dict

# Replace 'folder_path' with the path to your folder containing JSON files
folder_path = 'Resources'
merged_dictionary = merge_json_files(folder_path)


{'abbiategrasso': [' ', '001 - TRIBUNALE', '002 - GIUDICE DI PACE', '003 - AA010'], 'abbiategrasso  ut dpmi1': [' ', '001 - TRIBUNALE', '002 - GIUDICE DI PACE'], 'acireale': [' ', '001 - TRIBUNALE DI ACIREALE', '002 - GIUDICE DI PACE ACIREALE', '003 - GIUDICE DI PACE TRECASTAGNI'], 'acireale  ut dpct': [' ', '001 - TRIBUNALE DI ACIREALE', '002 - GIUDICE DI PACE ACIREALE', '003 - GIUDICE DI PACE TRECASTAGNI'], 'acqui terme': [' ', '001 - PRETURA ACQUI TERME', '002 - TRIBUNALE ACQUI TERME', '003 - GIUDICE DI PACE ACQUI TERME', '004 - GIUDICE DI PACE DI OVADA'], 'acqui terme  ut dpal': [' ', '001 - PRETURA ACQUI TERME', '002 - TRIBUNALE ACQUI TERME', '003 - GIUDICE DI PACE ACQUI TERME', '004 - GIUDICE DI PACE DI OVADA'], 'adria': [' ', '001 - PRETURA DI ADRIA', '002 - GIUDICE DI PACE DI ADRIA', '003 - TRIB.LE ROVIGO - SEZ. DI ADRIA'], 'adria  ut dpro': [' ', '001 - PRETURA DI ADRIA', '002 - GIUDICE DI PACE DI ADRIA', '003 - TRIB.LE ROVIGO - SEZ. DI ADRIA'], 'agrigento': [' ', '001 - TRIBU

In [5]:
def filter_empty_strings(dictionary):
    filtered_dict = {}
    for key, value in dictionary.items():
        if isinstance(value, list):
            filtered_dict[key] = [item for item in value if item.strip()]
        else:
            filtered_dict[key] = value
    return filtered_dict

# Assuming merged_dictionary is the dictionary obtained from merging JSON files
merged_dictionary_filtered = filter_empty_strings(merged_dictionary)
merged_dictionary_filtered

{'abbiategrasso': ['001 - TRIBUNALE', '002 - GIUDICE DI PACE', '003 - AA010'],
 'abbiategrasso  ut dpmi1': ['001 - TRIBUNALE', '002 - GIUDICE DI PACE'],
 'acireale': ['001 - TRIBUNALE DI ACIREALE',
  '002 - GIUDICE DI PACE ACIREALE',
  '003 - GIUDICE DI PACE TRECASTAGNI'],
 'acireale  ut dpct': ['001 - TRIBUNALE DI ACIREALE',
  '002 - GIUDICE DI PACE ACIREALE',
  '003 - GIUDICE DI PACE TRECASTAGNI'],
 'acqui terme': ['001 - PRETURA ACQUI TERME',
  '002 - TRIBUNALE ACQUI TERME',
  '003 - GIUDICE DI PACE ACQUI TERME',
  '004 - GIUDICE DI PACE DI OVADA'],
 'acqui terme  ut dpal': ['001 - PRETURA ACQUI TERME',
  '002 - TRIBUNALE ACQUI TERME',
  '003 - GIUDICE DI PACE ACQUI TERME',
  '004 - GIUDICE DI PACE DI OVADA'],
 'adria': ['001 - PRETURA DI ADRIA',
  '002 - GIUDICE DI PACE DI ADRIA',
  '003 - TRIB.LE ROVIGO - SEZ. DI ADRIA'],
 'adria  ut dpro': ['001 - PRETURA DI ADRIA',
  '002 - GIUDICE DI PACE DI ADRIA',
  '003 - TRIB.LE ROVIGO - SEZ. DI ADRIA'],
 'agrigento': ['001 - TRIBUNALE AGRI

In [7]:
def filter_keys_with_word(dictionary, word):
    filtered_dict = {key: value for key, value in dictionary.items() if word in key}
    return filtered_dict

# Assuming merged_dictionary is your dictionary
filtered_dictionary = filter_keys_with_word(merged_dictionary_filtered, 'ut')
print(filtered_dictionary)

{'abbiategrasso  ut dpmi1': ['001 - TRIBUNALE', '002 - GIUDICE DI PACE'], 'acireale  ut dpct': ['001 - TRIBUNALE DI ACIREALE', '002 - GIUDICE DI PACE ACIREALE', '003 - GIUDICE DI PACE TRECASTAGNI'], 'acqui terme  ut dpal': ['001 - PRETURA ACQUI TERME', '002 - TRIBUNALE ACQUI TERME', '003 - GIUDICE DI PACE ACQUI TERME', '004 - GIUDICE DI PACE DI OVADA'], 'adria  ut dpro': ['001 - PRETURA DI ADRIA', '002 - GIUDICE DI PACE DI ADRIA', '003 - TRIB.LE ROVIGO - SEZ. DI ADRIA'], 'agrigento  ut dpag': ['001 - TRIBUNALE AGRIGENTO', '002 - GIUDICE DI PACE ARAGONA', '003 - GIUDICE DI PACE DI AGRIGENTO', '004 - GIUDICE DI PACE DI BIVONA', '005 - GIUDICE DI PACE DI CAMMARATA', '006 - GIUDICE DI PACE DI CASTELTERMI', '007 - GIUD. PACE CATTOLICA ERACLEA', '008 - PRETURA AGRIGENTO', '009 - PRETURA DI CAMMARATA', '010 - PRETURA DI CASTELTERMINI', '011 - PRETURA DI ARAGONA', '012 - PRETURA DI BIVONA', '013 - PRETURA DI FAVARA', '014 - PRETURA DI CATTOLICA ERACLEA', '015 - GIUDICE PACE DI FAVARA', '016 - 

In [12]:
save_dict_to_json(filtered_dictionary, f'Resources/enti_completi_ut.json')

In [8]:
len(filtered_dictionary)


355